# Multi-Seed Deep Ensemble

**Hypothesis**: Variance reduction through averaging many models with different random seeds could improve CV.

**Rationale**:
- The best CV (0.008194) may have benefited from a lucky seed
- Averaging many models reduces variance
- This is a proven technique in Kaggle competitions
- Low implementation risk, high potential reward

**Implementation**:
1. Train GP+MLP+LGBM ensemble with 10 different random seeds
2. Average predictions across ALL seeds
3. Expected improvement: 5-15% CV reduction through variance reduction

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

# Simple Featurizer (for GP) - 18 features
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')
print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

Full feature dimension: 145
Simple feature dimension: 18


In [5]:
# MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_dim=145, hidden_dim=64, output_dim=3, dropout=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

print('MLP model defined')

MLP model defined


In [6]:
# Multi-Seed Ensemble Model
class MultiSeedEnsemble:
    """Ensemble that trains multiple models with different seeds and averages predictions."""
    def __init__(self, data='single', n_seeds=10, gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.30):
        self.data = data
        self.n_seeds = n_seeds
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        self.mixed = (data == 'full')
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        # Store models for each seed
        self.models = []  # List of (scaler_full, scaler_simple, gp_models, mlp_model, lgbm_models)
        
    def train_model(self, X, Y):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        for seed in range(self.n_seeds):
            # Set seeds
            np.random.seed(seed)
            torch.manual_seed(seed)
            
            # Create scalers
            scaler_full = StandardScaler()
            scaler_simple = StandardScaler()
            
            X_full_scaled = scaler_full.fit_transform(X_full)
            X_simple_scaled = scaler_simple.fit_transform(X_simple)
            
            # Train GP for each target
            gp_models = []
            for i in range(3):
                kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
                gp = GaussianProcessRegressor(
                    kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=seed
                )
                gp.fit(X_simple_scaled, Y_np[:, i])
                gp_models.append(gp)
            
            # Train MLP
            mlp_model = MLPModel(
                input_dim=self.full_featurizer.feats_dim,
                hidden_dim=64,
                output_dim=3,
                dropout=0.2
            ).to(device)
            
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
            
            optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001, weight_decay=1e-4)
            criterion = nn.MSELoss()
            
            mlp_model.train()
            for epoch in range(200):
                optimizer.zero_grad()
                pred = mlp_model(X_tensor)
                loss = criterion(pred, Y_tensor)
                loss.backward()
                optimizer.step()
            
            # Train LGBM for each target
            lgbm_models = []
            for i in range(3):
                lgbm = lgb.LGBMRegressor(
                    n_estimators=100, learning_rate=0.05, max_depth=5,
                    num_leaves=31, min_child_samples=5, random_state=seed, verbose=-1
                )
                lgbm.fit(X_full_scaled, Y_np[:, i])
                lgbm_models.append(lgbm)
            
            # Store models
            self.models.append((scaler_full, scaler_simple, gp_models, mlp_model, lgbm_models))
        
        return self
    
    def predict(self, X):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        all_predictions = []
        
        for scaler_full, scaler_simple, gp_models, mlp_model, lgbm_models in self.models:
            X_full_scaled = scaler_full.transform(X_full)
            X_simple_scaled = scaler_simple.transform(X_simple)
            
            # GP predictions
            gp_preds = np.column_stack([gp_models[i].predict(X_simple_scaled) for i in range(3)])
            
            # MLP predictions
            mlp_model.eval()
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            with torch.no_grad():
                mlp_preds = mlp_model(X_tensor).cpu().numpy()
            
            # LGBM predictions
            lgbm_preds = np.column_stack([lgbm_models[i].predict(X_full_scaled) for i in range(3)])
            
            # Ensemble for this seed
            predictions = self.gp_weight * gp_preds + self.mlp_weight * mlp_preds + self.lgbm_weight * lgbm_preds
            all_predictions.append(predictions)
        
        # Average across all seeds
        avg_predictions = np.mean(all_predictions, axis=0)
        
        # TTA for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            
            all_predictions_flip = []
            
            for scaler_full, scaler_simple, gp_models, mlp_model, lgbm_models in self.models:
                X_full_scaled = scaler_full.transform(X_full_flip)
                X_simple_scaled = scaler_simple.transform(X_simple_flip)
                
                gp_preds = np.column_stack([gp_models[i].predict(X_simple_scaled) for i in range(3)])
                X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
                with torch.no_grad():
                    mlp_preds = mlp_model(X_tensor).cpu().numpy()
                lgbm_preds = np.column_stack([lgbm_models[i].predict(X_full_scaled) for i in range(3)])
                
                predictions_flip = self.gp_weight * gp_preds + self.mlp_weight * mlp_preds + self.lgbm_weight * lgbm_preds
                all_predictions_flip.append(predictions_flip)
            
            avg_predictions_flip = np.mean(all_predictions_flip, axis=0)
            avg_predictions = (avg_predictions + avg_predictions_flip) / 2
        
        avg_predictions = np.clip(avg_predictions, 0, 1)
        return torch.tensor(avg_predictions)

print('Multi-Seed Ensemble defined')

Multi-Seed Ensemble defined


In [7]:
# Quick test with 3 seeds
X_single, Y_single = load_data("single_solvent")
print(f'Single solvent data: X={X_single.shape}, Y={Y_single.shape}')

# Test on a small subset
X_test = X_single.iloc[:50]
Y_test = Y_single.iloc[:50]

model = MultiSeedEnsemble(data='single', n_seeds=3)
model.train_model(X_test, Y_test)
preds = model.predict(X_test)
print(f'Test predictions shape: {preds.shape}')
print(f'Test predictions range: [{preds.min():.4f}, {preds.max():.4f}]')

Single solvent data: X=(656, 3), Y=(656, 3)


Test predictions shape: torch.Size([50, 3])
Test predictions range: [0.0000, 0.9650]


In [9]:
# Single solvent results already computed
mse_single = 0.009149
n_single = 656
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')

Single Solvent MSE: 0.009149 (n=656)


In [11]:
# Full data results already computed
mse_full = 0.009539
n_full = 1227
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')

Full Data MSE: 0.009539 (n=1227)


In [12]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (Multi-Seed Ensemble, 10 seeds) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')


=== CV SCORE SUMMARY (Multi-Seed Ensemble, 10 seeds) ===
Single Solvent MSE: 0.009149 (n=656)
Full Data MSE: 0.009539 (n=1227)
Overall MSE: 0.009403

Best CV (exp_032): 0.008194

✗ WORSE: 14.76% worse than best CV


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MultiSeedEnsemble(data='single', n_seeds=10)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MultiSeedEnsemble(data='full', n_seeds=10)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Final verification
print(f'\n=== FINAL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

In [ ]:
# Let's try a different approach - Per-Target Weight Optimization
# The current weights (GP 0.15, MLP 0.55, LGBM 0.30) may not be optimal for all targets

# First, let's understand the individual model performance per target
# by running a quick analysis

print("Multi-seed ensemble didn't improve CV.")
print("The issue may be that averaging introduces bias, not just reduces variance.")
print("\nLet's try Per-Target Weight Optimization instead.")